In [1]:
import pandas as pd

In [10]:
df = pd.read_excel("data/bdd.xlsx")

In [21]:
df

,cis,nom_commercial,dci,atc
0,61266250,A,"CONCENTRAT DE VITAMINE A SYNTHÉTIQUE, FORME HU...",NaN
1,62869109,A,"CONCENTRAT DE VITAMINE A SYNTHÉTIQUE, FORME HU...",NaN
2,69103878,A.D.N. BOIRON,A.D.N. POUR PRÉPARATIONS HOMÉOPATHIQUES,NaN
3,61876780,ABACAVIR ARROW,SULFATE D'ABACAVIR,J05AF06
4,63797011,ABACAVIR SANDOZ,ABACAVIR,J05AF06
...,...,...,...,...
15818,64949486,ZYRTECSET,CÉTIRIZINE (DICHLORHYDRATE DE),NaN
15819,67337081,ZYTIGA,ACÉTATE D'ABIRATÉRONE,L02BX03
15820,63095061,ZYVOXID,LINÉZOLIDE,J01XX08
15821,63283095,ZYVOXID,LINÉZOLIDE,J01XX08


In [12]:
df["dci"].unique().shape

(3972,)

In [23]:
import re

def garder_premier_mot(m):
    return m.split(" ")[0].strip()


df["nom_commercial"] = df["nom_commercial"].apply(garder_premier_mot)


In [24]:
df

,cis,nom_commercial,dci,atc
0,61266250,A,"CONCENTRAT DE VITAMINE A SYNTHÉTIQUE, FORME HU...",NaN
1,62869109,A,"CONCENTRAT DE VITAMINE A SYNTHÉTIQUE, FORME HU...",NaN
2,69103878,A.D.N.,A.D.N. POUR PRÉPARATIONS HOMÉOPATHIQUES,NaN
3,61876780,ABACAVIR,SULFATE D'ABACAVIR,J05AF06
4,63797011,ABACAVIR,ABACAVIR,J05AF06
...,...,...,...,...
15818,64949486,ZYRTECSET,CÉTIRIZINE (DICHLORHYDRATE DE),NaN
15819,67337081,ZYTIGA,ACÉTATE D'ABIRATÉRONE,L02BX03
15820,63095061,ZYVOXID,LINÉZOLIDE,J01XX08
15821,63283095,ZYVOXID,LINÉZOLIDE,J01XX08


In [25]:
df_unique = df.drop_duplicates(subset=['nom_commercial'], keep='first')


In [26]:
df_unique

,cis,nom_commercial,dci,atc
0,61266250,A,"CONCENTRAT DE VITAMINE A SYNTHÉTIQUE, FORME HU...",NaN
2,69103878,A.D.N.,A.D.N. POUR PRÉPARATIONS HOMÉOPATHIQUES,NaN
3,61876780,ABACAVIR,SULFATE D'ABACAVIR,J05AF06
6,68257528,ABACAVIR/LAMIVUDINE,CHLORHYDRATE D'ABACAVIR MONOHYDRATÉ; LAMIVUDINE,NaN
15,63152345,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,LAMIVUDINE; SULFATE D'ABACAVIR; ZIDOVUDINE,J05AR04
...,...,...,...,...
15810,60841335,ZYPREXA,OLANZAPINE,N05AH03
15816,65016707,ZYRTEC,DICHLORHYDRATE DE CÉTIRIZINE,NaN
15817,62636293,ZYRTECSET,CÉTIRIZINE (DICHLORHYDRATE DE),NaN
15819,67337081,ZYTIGA,ACÉTATE D'ABIRATÉRONE,L02BX03


In [27]:
df_unique.to_excel("ess.xlsx")